In [1]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras_core as keras
import keras_nlp
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

print("TensorFlow version:", tf.__version__)
print("KerasNLP version:", keras_nlp.__version__)

2024-11-12 18:55:05.625906: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 18:55:05.684440: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 18:55:05.712844: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731434105.736235    7963 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731434105.742855    7963 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 18:55:05.779140: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Using TensorFlow backend


/home/alexis/Disaster_tweets/venvTweets/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TensorFlow version: 2.18.0
KerasNLP version: 0.17.0


In [23]:
df_train = pd.read_csv("/home/alexis/nlp-getting-started/train.csv")
df_test = pd.read_csv("/home/alexis/nlp-getting-started/test.csv")

df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
print("Répertoire courant:", os.getcwd())

Répertoire courant: /home/alexis/Disaster_tweets


In [14]:
def replace_text(df):
    # Colonnes à traiter
    text_columns = ["id", "keyword", "location"]

    # Remplacer les valeurs nulles dans 'id', 'keyword' et 'location' avec la première valeur non nulle parmi ces colonnes
    for col in text_columns:
        df[col] = df[col].fillna(df[text_columns].bfill(axis=1).iloc[:, 0])
        
    return df

def replace_target(df):
    # Remplacer les valeurs manquantes dans 'target' avec la première valeur non nulle parmi 'keyword', 'location', 'text'
    df['target'] = df['target'].fillna(df[['keyword', 'location', 'text']].bfill(axis=1).iloc[:, 0])
    
    # Supprimer les lignes où le target est égal à l'une des autres colonnes (idéalement pas nécessaire avec les valeurs de remplacement)
    df.loc[df['target'] == df['keyword'], 'keyword'] = np.nan
    df.loc[df['target'] == df['location'], 'location'] = np.nan
    df.loc[df['target'] == df['text'], 'text'] = np.nan
    
    return df


def replace_location(df):
    # Assurez-vous que la colonne 'location' est de type chaîne
    df['location'] = df['location'].astype(str)
    
    # Remplacer la valeur dans 'location' si elle contient un '?'
    df['location'] = df['location'].where(~df['location'].str.contains(r'\?', na=False), np.nan)
    return df


def drop_null_target(df):
    # Supprimer les lignes où la colonne 'target' est manquante
    df = df.dropna(subset=['target'])
    return df

def drop_null_text(df):
    # Supprimer les lignes où la colonne 'text' est manquante
    df = df.dropna(subset=['text'])
    return df

def reset_ids(df):
    # Réinitialiser les IDs (par exemple, en numérotant les lignes)
    df['id'] = np.arange(1, len(df) + 1)
    return df

def reset_ids(df):
    # Réinitialiser les IDs (par exemple, en numérotant les lignes)
    df['id'] = np.arange(1, len(df) + 1)
    return df


In [15]:
def tri(df):
    df = replace_text(df)  # Remplacer le texte null
    df = replace_target(df)  # Remplacer le target null
    df = replace_location(df)  # Supprimer le contenu de 'location' si '?'
    df = drop_null_target(df)  # Supprimer les lignes où 'target' est manquant
    df = drop_null_text(df)  # Supprimer les lignes où 'text' est manquant
    df = reset_ids(df)  # Réinitialiser les IDs correctement
    return df

def tri_test(df):
    df = replace_text(df)  # Remplacer le texte null
    df = replace_location(df)  # Supprimer le contenu de 'location' si '?'
    df = drop_null_text(df)  # Supprimer les lignes où 'text' est manquant
    df = df[df['id'].notnull()]  # Supprimer les lignes où 'id' est manquant
    return df

In [26]:
train= tri(df_train)
test= tri_test(df_test)

/tmp/ipykernel_7963/2354450672.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(df[text_columns].bfill(axis=1).iloc[:, 0])
/tmp/ipykernel_7963/2354450672.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(df[text_columns].bfill(axis=1).iloc[:, 0])
/tmp/ipykernel_7963/2354450672.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_opt

In [33]:

train.head()

,id,keyword,location,text,target,length
0,1,NaN,nan,Our Deeds are the Reason of this #earthquake M...,1,69
1,2,4.0,4.0,Forest fire near La Ronge Sask. Canada,1,38
2,3,5.0,5.0,All residents asked to 'shelter in place' are ...,1,133
3,4,6.0,6.0,"13,000 people receive #wildfires evacuation or...",1,65
4,5,7.0,7.0,Just got sent this photo from Ruby #Alaska as ...,1,88


In [28]:
train["length"] = train["text"].apply(lambda x : len(x))
df_test["length"] = df_test["text"].apply(lambda x : len(x))

print("Train Length Stat")
print(train["length"].describe())

print("Test Length Stat")
print(df_test["length"].describe())

Train Length Stat
count    7613.000000
mean      101.037436
std        33.781325
min         7.000000
25%        78.000000
50%       107.000000
75%       133.000000
max       157.000000
Name: length, dtype: float64
Test Length Stat
count    3263.000000
mean      102.108183
std        33.972158
min         5.000000
25%        78.000000
50%       109.000000
75%       134.000000
max       151.000000
Name: length, dtype: float64


In [20]:
BATCH_SIZE = 32
NUM_TRAINING_EXAMPLES = df_train.shape[0]
TRAIN_SPLIT = 0.8
VAL_SPLIT = 0.2
STEPS_PER_EPOCH = int(NUM_TRAINING_EXAMPLES)*TRAIN_SPLIT // BATCH_SIZE

EPOCHS = 2
AUTO = tf.data.experimental.AUTOTUNE

In [34]:
from sklearn.model_selection import train_test_split

X = df_train["text"]
y = df_train["target"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=VAL_SPLIT, random_state=42)

X_test = df_test["text"]

In [35]:
# Load a DistilBERT model.
preset= "distil_bert_base_en_uncased"

# Use a shorter sequence length.
preprocessor = keras_nlp.models.DistilBertPreprocessor.from_preset(preset,
                                                                   sequence_length=160,
                                                                   name="preprocessor_4_tweets"
                                                                  )

# Pretrained classifier.
classifier = keras_nlp.models.DistilBertClassifier.from_preset(preset,
                                                               preprocessor = preprocessor, 
                                                               num_classes=2)

classifier.summary()

100%|██████████| 515/515 [00:00<00:00, 697kB/s]


100%|██████████| 580/580 [00:00<00:00, 251kB/s]


100%|██████████| 226k/226k [00:00<00:00, 536kB/s]
2024-11-12 23:32:19.170011: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2024-11-12 23:32:19.281643: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 14801376 exceeds 10% of free system memory.
2024-11-12 23:32:20.249097: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
2024-11-12 23:32:20.736659: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
2024-11-12 23:32:21.356440: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.


100%|██████████| 253M/253M [00:53<00:00, 4.97MB/s] 
2024-11-12 23:33:19.061322: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.


Preprocessor: "preprocessor_4_tweets"

...

Model: "distil_bert_text_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ distil_bert_backbone          │ (None, None, 768)         │      66,362,880 │ padding_mask[0][0],        │
│ (DistilBertBackbone)          │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ get_item (GetItem)            │ (None, 768)               │               0 │ distil_bert_backbone[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pooled_dense (Dense)          │ (None, 768)               │         590,592 │ get_item[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output_dropout (Dropout)      │ (None, 768)               │               0 │ pooled_dense[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ logits (Dense)                │ (None, 2)                 │           1,538 │ output_dropout[0][0]       │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 66,955,010 (255.41 MB)

 Trainable params: 66,955,010 (255.41 MB)

 Non-trainable params: 0 (0.00 B)

In [38]:
# Compile
classifier = tf.keras.Sequential([
    # Ajoutez ici vos couches de modèle (par exemple, tf.keras.layers.Dense, tf.keras.layers.Conv2D, etc.)
])

# Compilation du modèle
classifier.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  # ou 'binary_crossentropy' pour un modèle binaire
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Taux d'apprentissage
    metrics=["accuracy"]
)

# Entraînement du modèle
history = classifier.fit(
    x=X_train,
    y=y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val, y_val)
)

Epoch 1/2


ValueError: Sequential model sequential cannot be built because it has no layers. Call `model.add(layer)`.